In [1]:
from langgraph.graph import StateGraph
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.llms import HuggingFacePipeline
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from agent_state_lib import AgentState
from petshop_memory_loader import load_memory_nodes
from langchain_core.runnables import RunnableLambda

In [2]:

print("Loading memories...")
def load_memory_test():
    db_path = "../sqlite_test/petshopdb/petshop_database.db"
    user_id = "pepuso"
    embedded_question = "which dogs are from China?"

    memory_chain:RunnableLambda = load_memory_nodes()
    state = memory_chain.invoke({"db_path":db_path, "user_id": user_id, "embedded_question": embedded_question})

    print("Profile Memory:")    
    print(state["profile_memory"])

    print("Facts Memory:")
    print(state["facts_memory"])

    print("chat history:")
    print(state["chat_memory"])

load_memory_test()


Loading memories...
Loaded 4 messages for user_id pepuso
Profile Memory:
pepuso has a 1-year-old Dachshund named canela, described as energetic.
Facts Memory:
PEKINGESE Nationality: China. Origin: Likely has Lhasa Apso blood, but its origins are lost in the mists of over 4,000 years. For centuries, it was the favorite of the imperial family in Beijing and was brought to Europe in 1860 by British and French soldiers who found specimens in the looted Summer Palace. Description: Weight ranges from 2 to 8 kg. Broad head with wide spacing between the eyes; short but prominent nose; wrinkled muzzle; deep stop; tightly closed mouth that should not reveal teeth or tongue; large, dark, shiny, slightly protruding eyes; heart-shaped ears with long feathering; short, thick neck; tail curled over the back; strong bone structure; long, straight coat with extended mane and abundant feathering. All colors are accepted except liver and albino. Temperament: Sensitive, extraordinarily affectionate with i

In [3]:
model_path = "h:/ML_Models/_gemma/model/gemma-2b-it"  # local folder

tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path)
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=512)
llm = HuggingFacePipeline(pipeline=pipe)
print("Model loaded.")

prompt = ChatPromptTemplate.from_messages([
    ("system", "Answer based on Profile and Docs and Chat History. If you don't find relevant information, just say you don't know and use your knowledge. Be very concise."),
    ("human", "{question}\n\nDocs:\n{facts_memory}\n\nProfile:\n{profile_memory}\n\nChat History:\n{chat_memory}"),
])

chain = prompt | llm | StrOutputParser()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cpu
C:\Users\admin\AppData\Local\Temp\ipykernel_7292\414039441.py:6: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=pipe)


Model loaded.


In [4]:
def setup_questions(state: AgentState):
    print("collect question from user", state)
    #hardcoded for now
    state["embedded_question"] = "which dogs have small size?"
    state["question"] = "which dogs are good companion for mini dachshunds?"
    state["user_id"] =  "pepuso"
    state["db_path"] = "../sqlite_test/petshopdb/petshop_database.db"
    return state


def recover_memory(state: AgentState):
    print("recover facts here", state)
    
    memory_chain:RunnableLambda = load_memory_nodes()
    state = memory_chain.invoke(state)
    return state

def generate_answer(state: AgentState):
    print("generate answer here", state)
    answer = chain.invoke({
        "question": state["question"],
        "profile_memory": "\n".join(state["profile_memory"]),
        "facts_memory": "\n".join(state["facts_memory"]),
        "chat_memory": state["chat_memory"]
    })

    return {**state, "answer": answer}


In [5]:
graph = StateGraph(AgentState)
graph.add_node("setup",setup_questions)
graph.add_node("remember",recover_memory)
graph.add_node("reason", generate_answer)

graph.set_entry_point("setup")
graph.add_edge("setup","remember")
graph.add_edge("remember","reason")

graph.set_finish_point("reason")
agent = graph.compile()

In [6]:
response = agent.invoke({  
})
print(response["answer"])


collect question from user {}
recover facts here {'question': 'which dogs are good companion for mini dachshunds?', 'embedded_question': 'which dogs have small size?', 'db_path': '../sqlite_test/petshopdb/petshop_database.db', 'user_id': 'pepuso'}
Loaded 4 messages for user_id pepuso
generate answer here {'question': 'which dogs are good companion for mini dachshunds?', 'embedded_question': 'which dogs have small size?', 'db_path': '../sqlite_test/petshopdb/petshop_database.db', 'user_id': 'pepuso', 'chat_memory': 'Human: my puppy runs along the table very often, why?\nAI: our puppyâ€™s behavior likely stems from high energy levels and curiosity, which are common in young dogs. Tables offer elevation and visibility, making them an exciting playground â€” especially if the puppy hasnâ€™t yet learned boundaries.\nHuman: does having a companion may help her to be less active\nAI: A companion can help redirect her energy through play and social interaction, but it may not reduce her activi

RuntimeError: [enforce fail at alloc_cpu.cpp:121] data. DefaultCPUAllocator: not enough memory: you tried to allocate 5095653152 bytes.